In [1]:
## source : https://www.educative.io/courses/langchain-llm/runnables-and-expression-language

In [2]:
! which python

/Users/kunal21/opt/miniconda3/envs/v_agentic_kn/bin/python


### Runnables in LangChains

In [1]:
import dotenv
import os

dotenv.load_dotenv()
GROQ_KEY = os.getenv('GROQ_KEY')

In [2]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(model='llama3-8b-8192', #'deepseek-r1-distill-llama-70b',#llama-3.3-70b-versatile
               api_key=GROQ_KEY)

##### Runnables -   Callable unit of work
Common methods ; 
1. invoke() processes a single input and returns a single output, ideal for individual requests.
2. batch () allows us to process a list of inputs simultaneously and return a list of corresponding outputs, which is much more efficient for handling multiple requests at once.
3. stream() processes a single input but returns an output as a stream of chunks, which is useful for displaying real-time progress or handling very large outputs.

- LECL - langchain expressions language : for chaining

In [4]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1080b1fd0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1080b2ba0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
## simple chain example - input->promptTemplate->parser->output
## finding setiment as positive or negative 

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

sentiment_template = PromptTemplate(
    input_variables=["Feedback"],
    template="Determine sentiment of this Feedback as Positive or Negative\n{Feedback}"
)

user_feedback = "delivery was on time "

chain = sentiment_template | llm | StrOutputParser()

response = chain.invoke({'Feedback':user_feedback})
response

'I would classify this feedback as Positive. The phrase "delivery was on time" suggests that the customer is satisfied with the delivery process and the product was received as expected, which is a positive experience.'

##### Extending the chain using RunnableLambda
query -> chain1 -> (output) -> chain2 -> output -> chain3 -> parser 


In [12]:
from langchain_core.runnables import RunnableLambda

parse_template = PromptTemplate(
    input_variables=["raw_information"],
    template="parse and clean the following customer feedback for key information : \n {raw_information}"
)

sentiment_template = PromptTemplate(
    input_variables=["key_feedback"],
    template="Give one word output as Classify the follwoing key words of a feedback as Positive or Negative : \n {key_feedback}"
)

## RunnableLambda allows us to use custom functions as Runnable components. 
# This enables functions to be used in chains! To properly format the outputs, 
# we can create two simple lambda functions for our use case.

formated_parsed_output = RunnableLambda(lambda output:{'raw_information':output})
formated_classified_output = RunnableLambda(lambda output:{'key_feedback':output})

user_feedback = "The delivery was late, and the product was damaged when it arrived. However, the customer support team was very helpful in resolving the issue quickly."

chain = parse_template | llm | sentiment_template | llm | StrOutputParser()

feedback_sentiment = chain.invoke({"raw_information":user_feedback})

feedback_sentiment

'Positive'

In [30]:
### to print the output of middle commponents of chain

def print_output(key_name, output):
    print(key_name,' : ',output.content,'\n')
    return output.content


formated_parsed_output = RunnableLambda(lambda output:print_output("raw_information",output))
formated_classified_output = RunnableLambda(lambda output:print_output('key_feedback',output))

user_feedback = "The delivery was late, and the product was damaged when it arrived. However, the customer support team was very helpful in resolving the issue quickly."

chain = (parse_template 
        | llm  
        | formated_parsed_output 
        | sentiment_template 
        | llm 
        | formated_classified_output
        |StrOutputParser())

feedback_sentiment = chain.invoke({"raw_information":user_feedback})

feedback_sentiment


raw_information  :  Here is the key information parsed and cleaned from the customer feedback:

**Issue:**

* Delivery was late
* Product was damaged when it arrived

**Resolution:**

* Customer support team was helpful
* Issue was resolved quickly

**Overall sentiment:**

* Negative (due to late delivery and damaged product)
* Positive (due to helpful customer support team) 

key_feedback  :  Positive 



'Positive'

#### Conditional routing

In [35]:
## for any negative message - send apology msg to user
from langchain_core.runnables import RunnableLambda

def print_output(key_name, output):
    print(key_name,' : ',output.content,'\n')
    return output.content


parse_template = PromptTemplate(
    input_variables=["raw_information"],
    template="parse and clean the following customer feedback for key information : \n {raw_information}"
)

sentiment_template = PromptTemplate(
    input_variables=["key_feedback"],
    template="Give one word output as Classify the follwoing key words of a feedback as Positive or Negative : \n {key_feedback}"
)

thankyou_template = PromptTemplate(
    input_variables=["sentiment"],
    template="Given the feedback, draft a thank you message for the user and request them to leave a positive rating on our webpage:\n\n{sentiment}"
)

apology_template = PromptTemplate(
    input_variables=["sentiment"],
    template="Given the feedback, draft an apology message for the user and mention that their concern has been forwarded to the relevant department:\n\n{sentiment}"
)

## creating separate chains to be triggered on condition
thankyou_chain = thankyou_template | llm | StrOutputParser()
apology_chain = apology_template | llm | StrOutputParser()

def route(info):
    if 'positive' in info['sentiment'].lower():
        return thankyou_chain
    else:
        return apology_chain


formated_parsed_output = RunnableLambda(lambda output:print_output("raw_information",output))
formated_classified_output = RunnableLambda(lambda output:print_output('key_feedback',output))

user_feedback = "The delivery was late, and the product was damaged when it arrived. However, the customer support team was very helpful in resolving the issue quickly."

chain = (parse_template 
        | llm  
        | formated_parsed_output 
        | sentiment_template 
        | llm 
        | formated_classified_output
        |StrOutputParser())

## output of sentiment goes as input to trigger chain
feedback_sentiment = chain.invoke({"raw_information":user_feedback}) 

RunnableLambda(route).invoke({'sentiment':feedback_sentiment})

raw_information  :  Here is the parsed and cleaned key information from the customer feedback:

* **Issue:** Delivery was late and product was damaged.
* **Resolution:** Customer support team was very helpful in resolving the issue quickly.
* **Key Takeaway:** Despite the initial issue with delivery and product damage, the customer support team's quick resolution was a positive aspect.

Note: I removed the phrase "However" as it's a transition word and not essential information. I also combined the two negative points into one issue, and condensed the resolution into one sentence. 

key_feedback  :  Positive 



'Here\'s a draft thank you message:\n\n"Dear [User],\n\nWe\'re thrilled to hear that you had a positive experience with our [service/product]! We\'re grateful for your feedback and appreciate the time you took to share your thoughts with us.\n\nYour feedback is invaluable in helping us improve and provide the best possible experience for our users. We\'re happy to hear that you enjoyed [specific aspect of service/product] and we\'ll make sure to pass on your kind words to our team.\n\nAs a small token of our appreciation, we\'d be grateful if you could leave a positive rating on our webpage. Your feedback will help others make informed decisions and we\'re confident that it will make a big difference to our business.\n\nThank you again for your kind words and for choosing [Your Company Name]. We look forward to serving you again in the future.\n\nBest regards,\n[Your Name]"\n\nFeel free to customize it as per your needs!'